In [2]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

print(pd.__version__)
print(np.__version__)


1.5.3
1.26.4


## Team Summary Stats (`team_stats_final`)

Este dataframe resume el desempeño global de cada equipo en la temporada seleccionada (`SEASON`). Incluye récord, puntos a favor/en contra, métricas por partido, eficiencia básica y próximo rival.

**Variables:**

- `rank`  
- `season`  
- `team`  
- `wins`  
- `losses`  
- `games` – partidos jugados  
- `points_for` – puntos anotados (totales)  
- `points_against` – puntos recibidos (totales)  
- `point_diff` – diferencia de puntos (`points_for - points_against`)  
- `pf_per_game` – puntos a favor por partido  
- `pa_per_game` – puntos en contra por partido  
- `turnover_margin` – `takeaways - giveaways`  
- `first_downs` – primeros y dieces ofensivos totales  
- `third_down_pct` – porcentaje de conversiones en tercer down  
- `pct_scoring_drives` – porcentaje de series ofensivas que terminan en puntos  
- `next_opponent` – próximo rival del equipo (o `BYE / No game scheduled` / `Season complete`)


In [3]:
# Temporada que quieres analizar (puedes cambiarla)
SEASON = 2025
YEARS = [SEASON]


In [4]:
# 1) Schedules: info a nivel partido (scores, equipos, etc.)
schedules = nfl.import_schedules(YEARS)

# 2) Play-by-play: ahora SIN limitar columnas, para que traiga todo
pbp = nfl.import_pbp_data(YEARS)

# Opcional: filtrar solo temporada regular
if "game_type" in schedules.columns:
    schedules = schedules[schedules["game_type"] == "REG"]

if "season_type" in pbp.columns:
    pbp = pbp[pbp["season_type"] == "REG"]
elif "game_type" in pbp.columns:
    pbp = pbp[pbp["game_type"] == "REG"]



2025 done.
Downcasting floats.


In [5]:
def build_team_game_results(sched: pd.DataFrame) -> pd.DataFrame:
    # Nos quedamos solo con partidos que tienen marcador final
    required_cols = ["season", "game_id", "home_team", "away_team", "home_score", "away_score"]
    for c in required_cols:
        if c not in sched.columns:
            raise ValueError(f"Falta la columna {c} en schedules")

    sched = sched.dropna(subset=["home_score", "away_score"]).copy()

    # Fila para equipo local
    home = pd.DataFrame({
        "season": sched["season"],
        "game_id": sched["game_id"],
        "team": sched["home_team"],
        "points_for": sched["home_score"],
        "points_against": sched["away_score"],
        "win": np.where(sched["home_score"] > sched["away_score"], 1, 0),
        "loss": np.where(sched["home_score"] < sched["away_score"], 1, 0),
    })

    # Fila para equipo visitante
    away = pd.DataFrame({
        "season": sched["season"],
        "game_id": sched["game_id"],
        "team": sched["away_team"],
        "points_for": sched["away_score"],
        "points_against": sched["home_score"],
        "win": np.where(sched["away_score"] > sched["home_score"], 1, 0),
        "loss": np.where(sched["away_score"] < sched["home_score"], 1, 0),
    })

    games_team = pd.concat([home, away], ignore_index=True)
    return games_team


games_team = build_team_game_results(schedules)
games_team.head()


,season,game_id,team,points_for,points_against,win,loss
0,2025,2025_01_DAL_PHI,PHI,24.0,20.0,1,0
1,2025,2025_01_KC_LAC,LAC,27.0,21.0,1,0
2,2025,2025_01_TB_ATL,ATL,20.0,23.0,0,1
3,2025,2025_01_CIN_CLE,CLE,16.0,17.0,0,1
4,2025,2025_01_MIA_IND,IND,33.0,8.0,1,0


In [6]:
team_record = (
    games_team
    .groupby(["season", "team"], as_index=False)
    .agg(
        wins=("win", "sum"),
        losses=("loss", "sum"),
        points_for=("points_for", "sum"),
        points_against=("points_against", "sum"),
    )
)

team_record.head()


,season,team,wins,losses,points_for,points_against
0,2025,ARI,3,11,302.0,388.0
1,2025,ATL,5,9,281.0,341.0
2,2025,BAL,7,7,335.0,320.0
3,2025,BUF,10,4,411.0,324.0
4,2025,CAR,7,7,264.0,317.0


In [7]:
def build_team_turnover_margin(pbp_df: pd.DataFrame) -> pd.DataFrame:
    needed = ["season", "game_id", "posteam", "defteam", "interception", "fumble_lost"]
    missing = [c for c in needed if c not in pbp_df.columns]
    if missing:
        raise ValueError(f"Faltan columnas en pbp: {missing}")

    df = pbp_df.copy()

    # Definimos "turnover": intercepción o fumble perdido
    df["turnover_flag"] = (
        (df["interception"] == 1) |
        (df["fumble_lost"] == 1)
    ).astype(int)

    # Filtramos solo jugadas que son turnover
    pbp_tov = df.loc[df["turnover_flag"] == 1,
                     ["season", "game_id", "posteam", "defteam", "turnover_flag"]]

    # Giveaways: entregas de balón del equipo ofensivo
    giveaways = (
        pbp_tov
        .groupby(["season", "posteam"], as_index=False)["turnover_flag"]
        .count()
        .rename(columns={"posteam": "team", "turnover_flag": "giveaways"})
    )

    # Takeaways: recuperaciones del equipo defensivo
    takeaways = (
        pbp_tov
        .groupby(["season", "defteam"], as_index=False)["turnover_flag"]
        .count()
        .rename(columns={"defteam": "team", "turnover_flag": "takeaways"})
    )

    # Partimos del team_record para asegurar mismos equipos
    team_tov = pd.merge(
        team_record[["season", "team"]],
        giveaways,
        on=["season", "team"],
        how="left"
    )

    team_tov = pd.merge(
        team_tov,
        takeaways,
        on=["season", "team"],
        how="left"
    )

    team_tov[["giveaways", "takeaways"]] = (
        team_tov[["giveaways", "takeaways"]]
        .fillna(0)
        .astype(int)
    )

    team_tov["turnover_margin"] = team_tov["takeaways"] - team_tov["giveaways"]

    return team_tov


team_turnovers = build_team_turnover_margin(pbp)
team_turnovers.head()



,season,team,giveaways,takeaways,turnover_margin
0,2025,ARI,17,16,-1
1,2025,ATL,16,16,0
2,2025,BAL,19,17,-2
3,2025,BUF,17,19,2
4,2025,CAR,18,17,-1


In [8]:
team_stats = (
    team_record
    .merge(
        team_turnovers[["season", "team", "turnover_margin"]],
        on=["season", "team"],
        how="left"
    )
)

team_stats["turnover_margin"] = team_stats["turnover_margin"].fillna(0).astype(int)

# Solo la temporada que fijamos en SEASON
team_stats_season = (
    team_stats
    .loc[team_stats["season"] == SEASON]
    .reset_index(drop=True)
)

team_stats_season


,season,team,wins,losses,points_for,points_against,turnover_margin
0,2025,ARI,3,11,302.0,388.0,-1
1,2025,ATL,5,9,281.0,341.0,0
2,2025,BAL,7,7,335.0,320.0,-2
3,2025,BUF,10,4,411.0,324.0,2
4,2025,CAR,7,7,264.0,317.0,-1
5,2025,CHI,10,4,365.0,338.0,20
6,2025,CIN,4,10,314.0,437.0,-5
7,2025,CLE,3,11,226.0,332.0,-4
8,2025,DAL,6,7,407.0,420.0,-9
9,2025,DEN,12,2,342.0,261.0,2


In [9]:

def get_next_opponents(schedules: pd.DataFrame, season: int, teams: pd.Series) -> pd.DataFrame:

    # Filtramos por temporada
    sched_season = schedules[schedules["season"] == season].copy()

    # Si existe game_type, nos quedamos solo con temporada regular
    if "game_type" in sched_season.columns:
        sched_season = sched_season[sched_season["game_type"] == "REG"]

    # Partidos pendientes: sin marcador
    pending = sched_season[
        sched_season["home_score"].isna() & sched_season["away_score"].isna()
    ].copy()

    # Si no hay partidos pendientes, temporada terminada
    if pending.empty:
        return pd.DataFrame({
            "team": teams.values,
            "next_opponent": ["Season complete"] * len(teams)
        })

    # Tomamos el week más cercano (mínimo)
    next_week = pending["week"].min()
    next_week_games = pending[pending["week"] == next_week]

    # Construimos mapeo equipo -> próximo rival
    rows = []
    for _, g in next_week_games.iterrows():
        rows.append({"team": g["home_team"], "next_opponent": g["away_team"]})
        rows.append({"team": g["away_team"], "next_opponent": g["home_team"]})

    mapping = pd.DataFrame(rows)

    # Aseguramos que todos los equipos aparezcan (para marcar BYE si no tienen juego)
    all_teams = pd.DataFrame({"team": teams.values})
    opp_df = all_teams.merge(mapping, on="team", how="left")

    opp_df["next_opponent"] = opp_df["next_opponent"].fillna("BYE / No game scheduled")

    return opp_df


games_count = (
    games_team
    .groupby(["season", "team"], as_index=False)
    .agg(games=("game_id", "nunique"))
)

# Partimos de team_stats_season y agregamos "games"
team_stats_ext = (
    team_stats_season
    .merge(games_count, on=["season", "team"], how="left")
)

# ---------------------------------------------------
# 2) Diferencia de puntos y métricas por partido
# ---------------------------------------------------
team_stats_ext["point_diff"] = (
    team_stats_ext["points_for"] - team_stats_ext["points_against"]
)

team_stats_ext["pf_per_game"] = (
    team_stats_ext["points_for"] / team_stats_ext["games"]
)

team_stats_ext["pa_per_game"] = (
    team_stats_ext["points_against"] / team_stats_ext["games"]
)


pbp_season = pbp[pbp["season"] == SEASON].copy()
pbp_season = pbp_season[pbp_season["posteam"].notna()].copy()

# ---------- First downs ----------
if all(col in pbp_season.columns for col in ["first_down_rush", "first_down_pass", "first_down_penalty"]):
    pbp_season["first_down_total"] = (
        pbp_season["first_down_rush"].fillna(0)
        + pbp_season["first_down_pass"].fillna(0)
        + pbp_season["first_down_penalty"].fillna(0)
    )
elif "first_down" in pbp_season.columns:
    pbp_season["first_down_total"] = pbp_season["first_down"].fillna(0)
else:
    # Si no están las columnas, ponemos 0s
    pbp_season["first_down_total"] = 0

first_downs_team = (
    pbp_season
    .groupby(["season", "posteam"], as_index=False)
    .agg(first_downs=("first_down_total", "sum"))
    .rename(columns={"posteam": "team"})
)

# ---------- 3rd down % ----------
# Aseguramos que existan columnas (si no, las creamos en 0)
for col in ["third_down_converted", "third_down_failed"]:
    if col not in pbp_season.columns:
        pbp_season[col] = 0

third_down_team = (
    pbp_season
    .groupby(["season", "posteam"], as_index=False)[["third_down_converted", "third_down_failed"]]
    .sum()
)

third_down_team["third_down_attempts"] = (
    third_down_team["third_down_converted"] + third_down_team["third_down_failed"]
)

third_down_team["third_down_pct"] = np.where(
    third_down_team["third_down_attempts"] > 0,
    third_down_team["third_down_converted"] / third_down_team["third_down_attempts"],
    np.nan
)

third_down_team = (
    third_down_team
    .rename(columns={"posteam": "team"})
    [["season", "team", "third_down_pct", "third_down_attempts"]]
)


# ---------------------------------------------------
# 4) Merge de todas las métricas ofensivas
# ---------------------------------------------------
team_stats_ext = (
    team_stats_ext
    .merge(first_downs_team, on=["season", "team"], how="left")
    .merge(third_down_team, on=["season", "team"], how="left")
)

# ---------------------------------------------------
# 5) Próximo rival por equipo
# ---------------------------------------------------
next_opp_df = get_next_opponents(schedules, SEASON, team_stats_ext["team"])

team_stats_ext = team_stats_ext.merge(next_opp_df, on="team", how="left")

# ---------------------------------------------------
# 6) Ordenar, ranking y filtrar solo la temporada SEASON
# ---------------------------------------------------
team_stats_final = (
    team_stats_ext
    .loc[team_stats_ext["season"] == SEASON]
    .sort_values(by=["wins", "point_diff", "points_for"], ascending=[False, False, False])
    .reset_index(drop=True)
)

team_stats_final.insert(0, "rank", np.arange(1, len(team_stats_final) + 1))

team_stats_final


,rank,season,team,wins,losses,points_for,points_against,turnover_margin,games,point_diff,pf_per_game,pa_per_game,first_downs,third_down_pct,third_down_attempts,next_opponent
0,1,2025,DEN,12,2,342.0,261.0,2,14,81.0,24.428571,18.642857,277.0,0.403226,186.0,JAX
1,2,2025,SEA,11,3,405.0,242.0,1,14,163.0,28.928571,17.285714,276.0,0.379518,166.0,LA
2,3,2025,LA,11,3,420.0,261.0,9,14,159.0,30.000000,18.642857,328.0,0.377622,143.0,SEA
3,4,2025,NE,11,3,382.0,276.0,0,14,106.0,27.285714,19.714286,286.0,0.431138,167.0,BAL
4,5,2025,BUF,10,4,411.0,324.0,2,14,87.0,29.357143,23.142857,308.0,0.459770,174.0,CLE
5,6,2025,JAX,10,4,376.0,292.0,10,14,84.0,26.857143,20.857143,289.0,0.368715,179.0,DEN
6,7,2025,SF,10,4,344.0,293.0,-7,14,51.0,24.571429,20.928571,310.0,0.491803,183.0,IND
7,8,2025,LAC,10,4,315.0,284.0,0,14,31.0,22.500000,20.285714,278.0,0.463415,205.0,DAL
8,9,2025,CHI,10,4,365.0,338.0,20,14,27.0,26.071429,24.142857,296.0,0.440415,193.0,GB
9,10,2025,HOU,9,5,323.0,228.0,12,14,95.0,23.071429,16.285714,269.0,0.375000,192.0,LV


## Offensive Team Stats (`offense_final`)

Este dataframe concentra las estadísticas ofensivas de cada equipo a nivel temporada, combinando datos de play-by-play con el scoreboard oficial. Permite evaluar volumen, eficiencia aérea y terrestre, producción de TDs y jugadas explosivas.

**Variables:**

- `team`  
- `points_scored` – puntos totales anotados por el equipo  
- `points_per_game` – puntos por partido  
- `total_tds` – TDs ofensivos totales (pase + carrera)  

**Passing:**
- `pass_attempts` – intentos de pase (play-by-play, `pass_attempt`)  
- `completed_passes` – pases completos  
- `completion_pct` – porcentaje de pases completos  
- `yards_per_attempt` – yardas por intento de pase  
- `pass_yards` – yardas totales por pase  
- `pass_yards_per_game` – yardas por pase por partido  
- `pass_tds` – touchdowns por pase  

**Rushing:**
- `rush_yards` – yardas terrestres totales  
- `rush_yards_per_game` – yardas terrestres por partido  
- `rush_tds` – touchdowns por acarreo  

**Turnovers & chain moving:**
- `fumbles` – fumbles perdidos  
- `interceptions` – intercepciones lanzadas  
- `first_downs` – primeros y dieces ofensivos totales  

**Situacionales y explosivas:**
- `third_down_pct` – porcentaje de conversión en tercer down  
- `twenty_plus` – número de jugadas de 20+ yardas  
- `longest_play` – yardas de la jugada más larga ofensiva


In [10]:
# PBP filtrado a la temporada seleccionada
pbp_off = pbp[pbp["season"] == SEASON].copy()

# Nos quedamos solo con jugadas que tienen equipo ofensivo válido
pbp_off = pbp_off[pbp_off["posteam"].notna()].copy()

pbp_off.head()


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
1,40.0,2025_01_ARI_NO,2025090705,NO,ARI,REG,1,ARI,away,NO,...,0.0,0.0,-0.352700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.0,2025_01_ARI_NO,2025090705,NO,ARI,REG,1,ARI,away,NO,...,0.0,0.0,-0.190052,NaN,NaN,NaN,NaN,NaN,0.511128,-51.112808
3,85.0,2025_01_ARI_NO,2025090705,NO,ARI,REG,1,ARI,away,NO,...,1.0,0.0,1.317340,0.939998,4.750889,3.0,0.666726,0.43911,0.668940,33.105968
4,115.0,2025_01_ARI_NO,2025090705,NO,ARI,REG,1,ARI,away,NO,...,0.0,0.0,-1.694360,NaN,NaN,NaN,NaN,NaN,0.492038,50.796207
5,135.0,2025_01_ARI_NO,2025090705,NO,ARI,REG,1,ARI,away,NO,...,0.0,0.0,-1.284150,NaN,NaN,NaN,NaN,NaN,0.716846,-71.684570


In [11]:
# Aseguramos copiar el pbp de esa season y solo jugadas con posteam
pbp_off = pbp[pbp["season"] == SEASON].copy()
pbp_off = pbp_off[pbp_off["posteam"].notna()].copy()

# ---------------------------
# PASSING
# ---------------------------
pbp_off["pass_attempt_flag"] = pbp_off.get("pass_attempt", 0).fillna(0)
pbp_off["pass_complete_flag"] = pbp_off.get("complete_pass", 0).fillna(0)
pbp_off["pass_yards"] = pbp_off.get("passing_yards", 0).fillna(0)
pbp_off["pass_td_flag"] = pbp_off.get("pass_touchdown", 0).fillna(0)

# ---------------------------
# RUSHING
# ---------------------------
pbp_off["rush_attempt_flag"] = pbp_off.get("rush_attempt", 0).fillna(0)
pbp_off["rush_yards"] = pbp_off.get("rushing_yards", 0).fillna(0)
pbp_off["rush_td_flag"] = pbp_off.get("rush_touchdown", 0).fillna(0)

# ---------------------------
# TURNOVERS
# ---------------------------
pbp_off["fumble_lost_flag"] = pbp_off.get("fumble_lost", 0).fillna(0)
pbp_off["interception_flag"] = pbp_off.get("interception", 0).fillna(0)

# ---------------------------
# FIRST DOWNS
# ---------------------------
# 'first_down' ya es 1 si la ofensiva consiguió primer down en la jugada
pbp_off["first_down_flag"] = pbp_off.get("first_down", 0).fillna(0)

# ---------------------------
# THIRD DOWN
# ---------------------------
pbp_off["third_down_converted"] = pbp_off.get("third_down_converted", 0).fillna(0)
pbp_off["third_down_failed"] = pbp_off.get("third_down_failed", 0).fillna(0)
pbp_off["third_down_attempt"] = (
    pbp_off["third_down_converted"] + pbp_off["third_down_failed"]
)

# ---------------------------
# 20+ YARD PLAYS & LONGEST
# ---------------------------
pbp_off["yards_gained_clean"] = pbp_off.get("yards_gained", 0).fillna(0)
pbp_off["twenty_plus_flag"] = np.where(pbp_off["yards_gained_clean"] >= 20, 1, 0)


In [12]:
team_off_pbp = (
    pbp_off
    .groupby("posteam", as_index=False)
    .agg(
        # Passing
        pass_attempts=("pass_attempt_flag", "sum"),
        completed_passes=("pass_complete_flag", "sum"),
        pass_yards=("pass_yards", "sum"),
        pass_tds=("pass_td_flag", "sum"),

        # Rushing
        rush_yards=("rush_yards", "sum"),
        rush_tds=("rush_td_flag", "sum"),

        # Turnovers
        fumbles=("fumble_lost_flag", "sum"),
        interceptions=("interception_flag", "sum"),

        # First downs
        first_downs=("first_down_flag", "sum"),

        # Third down
        third_down_converted=("third_down_converted", "sum"),
        third_down_attempts=("third_down_attempt", "sum"),

        # Explosivas y longest play
        twenty_plus=("twenty_plus_flag", "sum"),
        longest_play=("yards_gained_clean", "max"),
    )
    .rename(columns={"posteam": "team"})
)


In [13]:
# Juegos por equipo (ya deberías tener games_team de antes)
games_count = (
    games_team
    .groupby("team", as_index=False)
    .agg(games=("game_id", "nunique"))
)

# Puntos del scoreboard oficial (team_stats_season ya tenía points_for)
points_team = team_stats_season[["team", "points_for"]].rename(
    columns={"points_for": "points_scored"}
)

# Merge de todo
offense_stats = (
    team_off_pbp
    .merge(games_count, on="team", how="left")
    .merge(points_team, on="team", how="left")
)

# Métricas por partido
offense_stats["points_per_game"] = offense_stats["points_scored"] / offense_stats["games"]
offense_stats["pass_yards_per_game"] = offense_stats["pass_yards"] / offense_stats["games"]
offense_stats["rush_yards_per_game"] = offense_stats["rush_yards"] / offense_stats["games"]

# Completions & Y/A
offense_stats["completion_pct"] = np.where(
    offense_stats["pass_attempts"] > 0,
    offense_stats["completed_passes"] / offense_stats["pass_attempts"],
    np.nan
)

offense_stats["yards_per_attempt"] = np.where(
    offense_stats["pass_attempts"] > 0,
    offense_stats["pass_yards"] / offense_stats["pass_attempts"],
    np.nan
)

# Total TDs ofensivos (pass + rush; si quieres incluir returns se puede sumar después)
offense_stats["total_tds"] = offense_stats["pass_tds"] + offense_stats["rush_tds"]

# 3rd down %
offense_stats["third_down_pct"] = np.where(
    offense_stats["third_down_attempts"] > 0,
    offense_stats["third_down_converted"] / offense_stats["third_down_attempts"],
    np.nan
)


In [14]:
offense_final = (
    offense_stats[
        [
            "team",
            "points_scored", "points_per_game",
            "total_tds",
            "pass_attempts", "completed_passes", "completion_pct",
            "yards_per_attempt",
            "pass_yards", "pass_yards_per_game", "pass_tds",
            "rush_yards", "rush_yards_per_game", "rush_tds",
            "fumbles", "interceptions",
            "first_downs",
            "third_down_pct",
            "twenty_plus",
            "longest_play",
        ]
    ]
    .sort_values(by="points_scored", ascending=False)
    .reset_index(drop=True)
)

offense_final


,team,points_scored,points_per_game,total_tds,pass_attempts,completed_passes,completion_pct,yards_per_attempt,pass_yards,pass_yards_per_game,pass_tds,rush_yards,rush_yards_per_game,rush_tds,fumbles,interceptions,first_downs,third_down_pct,twenty_plus,longest_play
0,DET,428.0,30.571429,51.0,485.0,315.0,0.649485,7.577320,3675.0,262.500000,30.0,1836.0,131.142857,21.0,3.0,5.0,280.0,0.383721,67,78.0
1,LA,420.0,30.000000,53.0,490.0,312.0,0.636735,7.595918,3722.0,265.857143,37.0,1770.0,126.428571,16.0,7.0,5.0,326.0,0.377622,65,88.0
2,BUF,411.0,29.357143,49.0,451.0,287.0,0.636364,7.383592,3330.0,237.857143,25.0,2219.0,158.500000,24.0,7.0,10.0,305.0,0.459770,66,64.0
3,DAL,407.0,29.071429,42.0,558.0,363.0,0.650538,7.222222,4030.0,287.857143,27.0,1691.0,120.785714,15.0,8.0,11.0,325.0,0.418605,57,74.0
4,SEA,405.0,28.928571,36.0,412.0,265.0,0.643204,8.368932,3448.0,246.285714,22.0,1581.0,112.928571,14.0,9.0,12.0,271.0,0.379518,62,67.0
5,IND,392.0,28.000000,44.0,475.0,298.0,0.627368,7.105263,3375.0,241.071429,20.0,1796.0,128.285714,24.0,5.0,10.0,294.0,0.415094,48,83.0
6,NE,382.0,27.285714,39.0,454.0,291.0,0.640969,7.883260,3579.0,255.642857,23.0,1714.0,122.428571,16.0,7.0,7.0,285.0,0.431138,61,72.0
7,JAX,376.0,26.857143,41.0,492.0,274.0,0.556911,6.530488,3213.0,229.500000,23.0,1690.0,120.714286,18.0,5.0,11.0,287.0,0.368715,51,71.0
8,CHI,365.0,26.071429,39.0,491.0,270.0,0.549898,6.515275,3199.0,228.500000,22.0,2126.0,151.857143,17.0,4.0,6.0,295.0,0.440415,57,65.0
9,GB,348.0,24.857143,40.0,452.0,286.0,0.632743,7.338496,3317.0,236.928571,24.0,1638.0,117.000000,16.0,5.0,6.0,278.0,0.502857,53,59.0


## Defensive Team Stats (`defense_final`)

Este dataframe resume el desempeño **defensivo** de cada equipo en la temporada seleccionada (`SEASON`).  
Las métricas provienen del play-by-play oficial (para producción defensiva y jugadas permitidas) y del scoreboard (para puntos permitidos).

Permite evaluar:  
- capacidad para limitar puntos y yardas  
- eficiencia contra el pase y la carrera  
- habilidad para forzar entregas de balón  
- fortaleza en terceros downs  
- susceptibilidad a jugadas explosivas  

---

### **Variables incluidas**

#### **Puntos y producción permitida**
- **`points_allowed`** – puntos totales permitidos (del scoreboard oficial)  
- **`points_allowed_per_game`** – puntos permitidos por partido  
- **`tds_allowed`** – TDs permitidos (pase + carrera)

#### **Defensa contra el pase**
- **`pass_yards_allowed`** – yardas aéreas permitidas  
- **`pass_yards_allowed_per_game`**  
- **`pass_tds_allowed`** – touchdowns por pase permitidos  

#### **Defensa contra la carrera**
- **`rush_yards_allowed`** – yardas terrestres permitidas  
- **`rush_yards_allowed_per_game`**  
- **`rush_tds_allowed`** – touchdowns terrestres permitidos  

#### **Takeaways (balones recuperados)**
- **`fumbles_recovered`** – fumbles recuperados por la defensa  
- **`interceptions`** – intercepciones generadas  

#### **Movimiento de cadenas permitido**
- **`first_downs_allowed`** – primeros y dieces permitidos  

#### **3rd Down Defense**
- **`third_down_stop_pct`** – porcentaje de stops defensivos en 3rd down  
  > Fórmula: *stops / intentos ofensivos de 3rd down*

#### **Jugadas explosivas permitidas**
- **`twenty_plus_allowed`** – jugadas permitidas de 20+ yardas  
- **`longest_play_allowed`** – jugada más larga permitida (en yardas)  




In [15]:
# PBP filtrado a la temporada y jugadas con defensa válida
pbp_def = pbp[pbp["season"] == SEASON].copy()
pbp_def = pbp_def[pbp_def["defteam"].notna()].copy()

# ---------------------------
# PASSING DEFENSE
# ---------------------------
pbp_def["pass_yards_allowed"] = pbp_def.get("passing_yards", 0).fillna(0)
pbp_def["pass_td_allowed_flag"] = pbp_def.get("pass_touchdown", 0).fillna(0)

# ---------------------------
# RUSHING DEFENSE
# ---------------------------
pbp_def["rush_yards_allowed"] = pbp_def.get("rushing_yards", 0).fillna(0)
pbp_def["rush_td_allowed_flag"] = pbp_def.get("rush_touchdown", 0).fillna(0)

# ---------------------------
# TAKEAWAYS DEFENSIVOS
# ---------------------------
pbp_def["fumble_lost_flag_off"] = pbp_def.get("fumble_lost", 0).fillna(0)
pbp_def["interception_flag_off"] = pbp_def.get("interception", 0).fillna(0)

# Para la defensa, cada fumble_lost de la ofensiva cuenta como fumble recuperado
pbp_def["fumbles_recovered_flag"] = pbp_def["fumble_lost_flag_off"]
pbp_def["interceptions_def_flag"] = pbp_def["interception_flag_off"]

# ---------------------------
# FIRST DOWNS PERMITIDOS
# ---------------------------
# 'first_down' = 1 si la ofensiva consiguió primer down en la jugada
pbp_def["first_down_allowed_flag"] = pbp_def.get("first_down", 0).fillna(0)

# ---------------------------
# THIRD DOWN STOPS (defensivos)
# ---------------------------
pbp_def["third_down_converted_off"] = pbp_def.get("third_down_converted", 0).fillna(0)
pbp_def["third_down_failed_off"] = pbp_def.get("third_down_failed", 0).fillna(0)
pbp_def["third_down_attempt_def"] = (
    pbp_def["third_down_converted_off"] + pbp_def["third_down_failed_off"]
)

# Para la defensa, un "stop" es cuando la ofensiva falla
pbp_def["third_down_stops_flag"] = pbp_def["third_down_failed_off"]

# ---------------------------
# 20+ PLAYS ALLOWED & LONGEST
# ---------------------------
pbp_def["yards_gained_clean"] = pbp_def.get("yards_gained", 0).fillna(0)
pbp_def["twenty_plus_allowed_flag"] = np.where(pbp_def["yards_gained_clean"] >= 20, 1, 0)


In [16]:
defense_team_raw = (
    pbp_def
    .groupby("defteam", as_index=False)
    .agg(
        # Passing
        pass_yards_allowed=("pass_yards_allowed", "sum"),
        pass_tds_allowed=("pass_td_allowed_flag", "sum"),

        # Rushing
        rush_yards_allowed=("rush_yards_allowed", "sum"),
        rush_tds_allowed=("rush_td_allowed_flag", "sum"),

        # Takeaways
        fumbles_recovered=("fumbles_recovered_flag", "sum"),
        interceptions=("interceptions_def_flag", "sum"),

        # First downs allowed
        first_downs_allowed=("first_down_allowed_flag", "sum"),

        # Third-down defense
        third_down_attempts_def=("third_down_attempt_def", "sum"),
        third_down_stops=("third_down_stops_flag", "sum"),

        # Explosivas y jugada más larga permitida
        twenty_plus_allowed=("twenty_plus_allowed_flag", "sum"),
        longest_play_allowed=("yards_gained_clean", "max"),
    )
    .rename(columns={"defteam": "team"})
)


In [17]:
# Juegos por equipo (ya lo tienes de antes)
games_count = (
    games_team
    .groupby("team", as_index=False)
    .agg(games=("game_id", "nunique"))
)

# Puntos permitidos del scoreboard oficial
points_allowed_team = team_stats_season[["team", "points_against"]].rename(
    columns={"points_against": "points_allowed"}
)

# Merge de todo
defense_stats = (
    defense_team_raw
    .merge(games_count, on="team", how="left")
    .merge(points_allowed_team, on="team", how="left")
)

# Métricas por partido
defense_stats["points_allowed_per_game"] = defense_stats["points_allowed"] / defense_stats["games"]
defense_stats["pass_yards_allowed_per_game"] = defense_stats["pass_yards_allowed"] / defense_stats["games"]
defense_stats["rush_yards_allowed_per_game"] = defense_stats["rush_yards_allowed"] / defense_stats["games"]

# TDs totales permitidos (pase + carrera)
defense_stats["tds_allowed"] = defense_stats["pass_tds_allowed"] + defense_stats["rush_tds_allowed"]

# 3rd down % of stops (defensivo)
defense_stats["third_down_stop_pct"] = np.where(
    defense_stats["third_down_attempts_def"] > 0,
    defense_stats["third_down_stops"] / defense_stats["third_down_attempts_def"],
    np.nan
)

# DataFrame final defensivo
defense_final = (
    defense_stats[
        [
            "team",
            "points_allowed", "points_allowed_per_game",
            "tds_allowed",
            "pass_yards_allowed", "pass_yards_allowed_per_game",
            "pass_tds_allowed",
            "rush_yards_allowed", "rush_yards_allowed_per_game",
            "rush_tds_allowed",
            "fumbles_recovered", "interceptions",
            "first_downs_allowed",
            "third_down_stop_pct",
            "twenty_plus_allowed",
            "longest_play_allowed",
        ]
    ]
    .sort_values(by="points_allowed", ascending=True)  # mejor defensa arriba
    .reset_index(drop=True)
)

defense_final


,team,points_allowed,points_allowed_per_game,tds_allowed,pass_yards_allowed,pass_yards_allowed_per_game,pass_tds_allowed,rush_yards_allowed,rush_yards_allowed_per_game,rush_tds_allowed,fumbles_recovered,interceptions,first_downs_allowed,third_down_stop_pct,twenty_plus_allowed,longest_play_allowed
0,HOU,228.0,16.285714,25.0,2763.0,197.357143,15.0,1298.0,92.714286,10.0,7.0,16.0,225.0,0.661017,44,56.0
1,SEA,242.0,17.285714,24.0,2981.0,212.928571,17.0,1282.0,91.571429,7.0,8.0,16.0,243.0,0.670103,41,65.0
2,LA,261.0,18.642857,26.0,3321.0,237.214286,21.0,1415.0,101.071429,5.0,8.0,13.0,266.0,0.638743,44,75.0
3,DEN,261.0,18.642857,24.0,3125.0,223.214286,14.0,1272.0,90.857143,10.0,5.0,9.0,245.0,0.680628,38,68.0
4,KC,268.0,19.142857,27.0,2972.0,212.285714,15.0,1387.0,99.071429,12.0,3.0,8.0,234.0,0.574713,43,71.0
5,PHI,271.0,19.357143,27.0,2945.0,210.357143,13.0,1762.0,125.857143,14.0,7.0,10.0,267.0,0.591623,45,77.0
6,NE,276.0,19.714286,31.0,3054.0,218.142857,24.0,1331.0,95.071429,7.0,6.0,8.0,254.0,0.625000,44,53.0
7,GB,281.0,20.071429,31.0,2920.0,208.571429,21.0,1407.0,100.500000,10.0,5.0,7.0,257.0,0.606557,38,45.0
8,LAC,284.0,20.285714,28.0,2674.0,191.000000,12.0,1463.0,104.500000,16.0,2.0,17.0,233.0,0.646341,36,52.0
9,JAX,292.0,20.857143,33.0,3310.0,236.428571,23.0,1208.0,86.285714,10.0,8.0,18.0,271.0,0.581006,43,61.0


#  NFL Stat Leaders — Temporada (`SEASON`)

Este módulo presenta a los **jugadores líderes** de la temporada en distintas métricas ofensivas clave, utilizando datos del *play-by-play* oficial extraídos mediante `nfl_data_py`.  
Para cada jugador se identifica su **equipo más reciente** dentro de la temporada, de modo que los líderes reflejan adecuadamente cambios de equipo o participación múltiple.

Las estadísticas agregadas incluyen:

- **Passing Yards** – Yardas totales por pase.  
- **Passing Touchdowns** – Touchdowns anotados por pase.  
- **Interceptions Thrown** – Intercepciones lanzadas por el mariscal.  
- **Rushing Yards** – Yardas por acarreo.  
- **Rushing Touchdowns** – Touchdowns por tierra.  
- **Receiving Yards** – Yardas por recepción.  
- **Receiving Touchdowns** – Touchdowns por recepción.  

Cada categoría cuenta con un dataframe separado con el **Top 10** de jugadores en esa métrica, mostrando:

- **Nombre del jugador**  
- **Equipo más reciente en la temporada**  
- **Valor estadístico correspondiente**  

Estos líderes permiten identificar fácilmente a los jugadores más productivos por tipo de contribución ofensiva, facilitando análisis para scouting, fantasy, reportes o dashboards.  
Las tablas se generan exclusivamente a partir de jugadas válidas del PBP y reflejan producción real en campo sin ajustes artificiales.


In [18]:
import pandas as pd
import numpy as np

# Filtramos solo la temporada actual
pbp_season = pbp[pbp["season"] == SEASON].copy()

# Aseguramos columnas numéricas presentes y sin NaN
for col in [
    "passing_yards", "pass_touchdown", "interception",
    "rushing_yards", "rush_touchdown",
    "receiving_yards"
]:
    if col not in pbp_season.columns:
        pbp_season[col] = 0
    pbp_season[col] = pbp_season[col].fillna(0)

# Helper para ordenar jugadas cronológicamente (para tomar el equipo más reciente)
def sort_for_latest(df: pd.DataFrame) -> pd.DataFrame:
    sort_cols = [c for c in ["game_date", "week", "game_id", "qtr", "down", "play_id"] if c in df.columns]
    if sort_cols:
        return df.sort_values(sort_cols)
    return df

# ======================================================
# PASSING STATS (QBs): yards, TDs, INTs lanzadas
# ======================================================

passing_plays = pbp_season[~pbp_season["passer_player_name"].isna()].copy()
passing_plays = sort_for_latest(passing_plays)

passing_stats = (
    passing_plays
    .groupby(["passer_player_id", "passer_player_name"], as_index=False)
    .agg(
        team=("posteam", "last"),                # equipo más reciente
        pass_yards=("passing_yards", "sum"),
        pass_tds=("pass_touchdown", "sum"),
        ints=("interception", "sum"),
    )
    .rename(columns={"passer_player_name": "player"})
)

leaders_pass_yards = (
    passing_stats
    .sort_values(by="pass_yards", ascending=False)
    .head(10)
    [["player", "team", "pass_yards"]]
    .reset_index(drop=True)
)

leaders_pass_tds = (
    passing_stats
    .sort_values(by="pass_tds", ascending=False)
    .head(10)
    [["player", "team", "pass_tds"]]
    .reset_index(drop=True)
)

leaders_ints = (
    passing_stats
    .sort_values(by="ints", ascending=False)
    .head(10)
    [["player", "team", "ints"]]
    .reset_index(drop=True)
)

# ======================================================
# RUSHING STATS: yards, TDs
# ======================================================

rushing_plays = pbp_season[~pbp_season["rusher_player_name"].isna()].copy()
rushing_plays = sort_for_latest(rushing_plays)

rushing_stats = (
    rushing_plays
    .groupby(["rusher_player_id", "rusher_player_name"], as_index=False)
    .agg(
        team=("posteam", "last"),
        rush_yards=("rushing_yards", "sum"),
        rush_tds=("rush_touchdown", "sum"),
    )
    .rename(columns={"rusher_player_name": "player"})
)

leaders_rush_yards = (
    rushing_stats
    .sort_values(by="rush_yards", ascending=False)
    .head(10)
    [["player", "team", "rush_yards"]]
    .reset_index(drop=True)
)

leaders_rush_tds = (
    rushing_stats
    .sort_values(by="rush_tds", ascending=False)
    .head(10)
    [["player", "team", "rush_tds"]]
    .reset_index(drop=True)
)

# ======================================================
# RECEIVING STATS: yards, TDs
# ======================================================

receiving_plays = pbp_season[~pbp_season["receiver_player_name"].isna()].copy()
receiving_plays = sort_for_latest(receiving_plays)

if "pass_touchdown" not in receiving_plays.columns:
    receiving_plays["pass_touchdown"] = 0
receiving_plays["pass_touchdown"] = receiving_plays["pass_touchdown"].fillna(0)

receiving_stats = (
    receiving_plays
    .groupby(["receiver_player_id", "receiver_player_name"], as_index=False)
    .agg(
        team=("posteam", "last"),
        rec_yards=("receiving_yards", "sum"),
        rec_tds=("pass_touchdown", "sum"),
    )
    .rename(columns={"receiver_player_name": "player"})
)

leaders_rec_yards = (
    receiving_stats
    .sort_values(by="rec_yards", ascending=False)
    .head(10)
    [["player", "team", "rec_yards"]]
    .reset_index(drop=True)
)

leaders_rec_tds = (
    receiving_stats
    .sort_values(by="rec_tds", ascending=False)
    .head(10)
    [["player", "team", "rec_tds"]]
    .reset_index(drop=True)
)


## Passing Yards Leaders (Top 10)

Jugadores con más yardas por pase en la temporada seleccionada.


In [19]:
leaders_pass_yards


,player,team,pass_yards
0,D.Prescott,DAL,3931.0
1,M.Stafford,LA,3722.0
2,J.Goff,DET,3672.0
3,P.Mahomes,KC,3587.0
4,D.Maye,NE,3567.0
5,S.Darnold,SEA,3433.0
6,J.Love,GB,3304.0
7,J.Allen,BUF,3276.0
8,B.Nix,DEN,3256.0
9,T.Lawrence,JAX,3210.0


## Passing TDs Leaders (Top 10)

Jugadores con más pases de touchdown en la temporada seleccionada.


In [20]:
leaders_pass_tds


,player,team,pass_tds
0,M.Stafford,LA,37.0
1,J.Goff,DET,29.0
2,D.Prescott,DAL,26.0
3,J.Allen,BUF,25.0
4,D.Maye,NE,23.0
5,B.Nix,DEN,23.0
6,J.Herbert,LAC,23.0
7,T.Lawrence,JAX,23.0
8,J.Love,GB,23.0
9,S.Darnold,SEA,22.0


## Interceptions Thrown Leaders (Top 10)

QBs con más intercepciones lanzadas en la temporada seleccionada.


In [21]:
leaders_ints


,player,team,ints
0,T.Tagovailoa,MIA,15.0
1,G.Smith,LV,14.0
2,J.Herbert,LAC,12.0
3,S.Darnold,SEA,11.0
4,T.Lawrence,JAX,11.0
5,J.McCarthy,MIN,11.0
6,P.Mahomes,KC,11.0
7,D.Prescott,DAL,10.0
8,J.Flacco,CIN,10.0
9,J.Allen,BUF,10.0


## Rushing Yards Leaders (Top 10)

Jugadores con más yardas terrestres acumuladas en la temporada seleccionada.


In [22]:
leaders_rush_yards


,player,team,rush_yards
0,J.Taylor,IND,1443.0
1,J.Cook,BUF,1415.0
2,D.Achane,MIA,1186.0
3,B.Robinson,ATL,1174.0
4,D.Henry,BAL,1125.0
5,J.Williams,DAL,1113.0
6,J.Gibbs,DET,1100.0
7,K.Williams,LA,1030.0
8,R.Dowdle,CAR,978.0
9,T.Etienne,JAX,949.0


## Rushing TDs Leaders (Top 10)

Jugadores con más touchdowns por acarreo en la temporada seleccionada.


In [23]:
leaders_rush_tds


,player,team,rush_tds
0,J.Taylor,IND,16.0
1,J.Gibbs,DET,13.0
2,J.Jacobs,GB,13.0
3,J.Allen,BUF,12.0
4,J.Williams,DAL,10.0
5,J.Cook,BUF,10.0
6,K.Williams,LA,10.0
7,D.Henry,BAL,10.0
8,C.McCaffrey,SF,9.0
9,J.Hurts,PHI,8.0


## Receiving Yards Leaders (Top 10)

Receptores con más yardas por recepción en la temporada seleccionada.


In [24]:
leaders_rec_yards


,player,team,rec_yards
0,J.Smith-Njigba,SEA,1541.0
1,P.Nacua,LA,1367.0
2,G.Pickens,DAL,1212.0
3,J.Chase,CIN,1147.0
4,A.St. Brown,DET,1140.0
5,T.McBride,ARI,1071.0
6,N.Collins,HOU,1001.0
7,C.Lamb,DAL,976.0
8,Z.Flowers,BAL,959.0
9,J.Williams,DET,928.0


## Receiving TDs Leaders (Top 10)

Receptores con más touchdowns por recepción en la temporada seleccionada.


In [25]:
leaders_rec_tds


,player,team,rec_tds
0,D.Adams,LA,14.0
1,A.St. Brown,DET,11.0
2,T.McBride,ARI,10.0
3,J.Smith-Njigba,SEA,9.0
4,T.Higgins,CIN,9.0
5,D.Goedert,PHI,9.0
6,G.Pickens,DAL,8.0
7,Q.Johnston,LAC,7.0
8,M.Pittman,IND,7.0
9,J.Jennings,SF,7.0
